<h1><center> Python project</center></h1>
<h2><center>Current car prices and other relevant parameters from bazos.cz</center></h2>
<h3><center>Daniel Brosko, Vojtěch Suchánek</center></h3>

Our goal is to web-scrape advertisements listed on website bazos.cz, which is currently one of the most used websites for selling used cars in Czech republic. It has more than 15 000 car adds daily. On the other hand, it has really poor search options, which pretty much complicates searching for desired car based on your parameters.

We are going to code algorithm, which will scan adds for the current day, pick those, which fulfill our conditions on date and car type and save their links. Then we will go to each link and save the text of the add. Then we will try to analyze the text of the add to find our parameters.

This approach might also allow for longer time period analysis in further steps - we would collect data periodically and investigate the trends in price changes, number of ads for selected car added during particular days, and more. However, since this project should be designed as one-time run, we decided to limit the data to only current date.

In [1]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import date, datetime, timedelta
import time
import numpy as np
import pandas as pd

The commented line below displays the version of packages so they can be used in requirements.txt.

In [3]:
#pip list

By the code in the following chunk, we checked that we are allowed to scrape particular parts of bazos.cz domain.

In [4]:
bots = requests.get('https://auto.bazos.cz/robots.txt')
#print(bots.text)

From the robots page we can see that our actions done in our projects are allowed, since we are not gonna use these search commands.

In the next chunk, we import two .py scripts where we defined the functions search_model, and n_days_search to filter and include only advertisements relevant to our preferences. More comments on the functions are printed few chunks below - where we print the documentation, but also by looking at .py scripts directly in GitHub.

In [5]:
from model_search import search_model
from n_days_search import n_days_search

In [6]:
# Now we run the functions we loaded from script, it will take some time, though
soup_list = search_model("octavia 3")

/Users/danielb/Desktop/IES/Python/Python-project/model_search.py:31: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 31 of the file /Users/danielb/Desktop/IES/Python/Python-project/model_search.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  no_of_adv_html = BeautifulSoup(page.text, 'html')
/Users/danielb/Desktop/IES/Python/Python-project/model_search.py:67: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and

Initial search for your model successfully finished.


In the next chunk, we filter for the ads added today + max 5 days old. If we compared the number for today (2022-08-30) there were 98 at the time, while the number of all ads for the same car-model input was 2022.

In [7]:
# Now we specify on how many past days we want to include in our search
list_of_offers_url = n_days_search(5, soup_list)

The number of found advertisements matching the criteria: 499 .


In [8]:
# Here we print the documentation for our functions we imported earlier.

help(search_model)

help(n_days_search)

Help on function search_model in module model_search:

search_model(user_input)
    Function search_model takes string input, hence it has to be in quotes ("").
    The input should be the name of the car model you would like to get results for,
    e.g. "octávia 3" - there should be no problem even when full Czech alphabet is used.
    
    Then the string is stripped of the characters that are not supposed to be in the search input,
    and if there are more than single word in the input, they are connected by '+' (plus) sign,
    since that is the format that bazos.cz use in their URLs.
    
    Then, the prepared string is paste into the common URL format that bazos.cz use.
    
    The very next step is obtaining the number of found advertisements for user's input from the html source code.
    This number of advertisements is used to select the proper length of the adv. tabs list that we will scrape.
    
    The function returns "soup_list" - the list of html codes for each adv.

Finally, we proceed to Data-mining part, where we extract the desired parameters - year of manufacture, year, and price. 

This is probably the most demanding part of the project - we need to extract the relevant data from unformated text. There is no official format of the text, so we tried to find a way how to extract this information from various formats. The results are not the best since sometimes it happen that our code is not able to recognize the unusual format of the parameter. In further steps, probably implementing some ML algorithm could improve the successful recognition significantly.

We save all of those parameters along with the URLs of particular advertisements. We created a class ResultTable that has two methods - "show_results" and "show_best" by which we can display the best recommended ads for our desired car model.

Hence, now we can take a look on potentially most interesting advertisements for us by following the URLs and checking the entire content of several ads instead of looking at "thousands" of them.

In [9]:
# DATA/TEXT MINING PART
def modify_text(text):
    """ Internal function modifying the text of the add to better suit for text mining. """
    return text.replace(" ", "").replace(".", "").replace("xxx", "000").replace("-", "")
def get_numbers_from_text(text):
    """ Internal function for finding all numbers in the text. """
    text = modify_text(text)
    pattern = '[.]?[\d]+[\.]?\d*(?:[eE][-+]?\d+)?'
    list_of_numbers = re.findall(pattern, text)
    return list_of_numbers
def find_years(numbers):
    """ Internal function picking numbers which might be years from all numbers in text. """
    numbers = [x for x in numbers if (float(x) > 1980) and (float(x) < 2023)]
    return numbers
def find_km(numbers):
    """ Internal function picking numbers which might be mileage from all numbers in text. """
    numbers = [x for x in numbers if (float(x) > 3000) and (float(x) < 500000)]
    return numbers
def get_context(text, list_of_tokens, year_dictionary = ['egistr', 'rv', 'RV', 'yrob', 'ýrob', 'prov', 'rok', 'Rok'], km_dictionary = ['km', 'Km', 'KM', 'ilomet', 'ajet', 'ájez', 'achom', 'atoč'], context_span=20):
    """ Internal function looking into surroundings of each year and mileage candidate and searching for parts of words defined in dictionaries. """
    #import re
    context = []
    year = 'No match'
    km = 'No match'
    for token in find_years(list_of_tokens):
        all_occurences_indices = [m.start() for m in re.finditer(token, text)]
        for index in all_occurences_indices:
            left_index = max(index - context_span, 0)
            right_index = min(index + context_span, len(text))
            substring = text[left_index: right_index].strip()
            for s in year_dictionary:
                year_find = [m.start() for m in re.finditer(s, substring)]
                if len(year_find) > 0:
                    year = token
    for token in find_km(list_of_tokens):
        all_occurences_indices = [m.start() for m in re.finditer(token, text)]
        for index in all_occurences_indices:
            left_index = max(index - context_span, 0)
            right_index = min(index + context_span, len(text))
            substring = text[left_index: right_index].strip()
            for s in km_dictionary:
                km_find = [m.start() for m in re.finditer(s, substring)]
                if len(km_find) > 0:
                    km = token
    return [year, km]

class ResultTable(pd.core.frame.DataFrame):
    def show_results(self, min_price = 0, max_price = 10000000, min_year = 1950, max_year = 2022, min_mileage = 0, max_mileage = 500000):
        temp_table = self[(self["mileage"] != "No match") & (self["year_of_manuf"] != "No match")]
        temp_table2 = temp_table[(pd.to_numeric(temp_table["price"]) > min_price) & (pd.to_numeric(temp_table["price"]) < max_price) &
                  (pd.to_numeric(temp_table["year_of_manuf"]) > min_year) & (pd.to_numeric(temp_table["year_of_manuf"]) < max_year) &
                    (pd.to_numeric(temp_table["mileage"]) > min_mileage) & (pd.to_numeric(temp_table["mileage"]) < max_mileage)].sort_values(by = "price")
        print(temp_table2)
    def show_best(self, n = 5, penalty = 5000):
        temp_table = self[(self["mileage"] != "No match") & (self["year_of_manuf"] != "No match")]
        temp_table = temp_table.assign(score=lambda x: ((2022 - pd.to_numeric(x.year_of_manuf))*penalty +
                                       pd.to_numeric(x.mileage)) / pd.to_numeric(x.price))
        temp_table = temp_table.sort_values(by = "score").head(n)
        print(temp_table)
def get_info(links):
    """ Get_info is a final function performing text mining and creating results in form of ResultTable class. """
    results_temp = []
    for i in links:
        print(i)
        add_page = requests.get(i)
        soup_add = BeautifulSoup(add_page.text, 'html')
        add = modify_text(soup_add.find('div', {'class':'popisdetail'}).get_text())
        price = soup_add.find('table').find_all('b')[-1].get_text()
        all_numbers = get_numbers_from_text(add)
        context_got = get_context(add, all_numbers)
        result = [i, context_got[0], context_got[1], price.replace(" ", "").replace("Kč", "")]
        results_temp.append(result)
        time.sleep(0.2)
    results = ResultTable(results_temp)
    results.columns = ['link', 'year_of_manuf', 'mileage', 'price']
    results = results[(results["price"] != "Dohodou") & (results["price"] != "Vtextu") & (results["price"] != "Nabídněte")]
    return results
result = get_info(list_of_offers_url)


pd.options.display.max_colwidth = 120
test = ResultTable(result)
test.show_results(min_price = 50000, max_price = 350000, min_year = 2013, max_year = 2018, min_mileage = 100000, max_mileage = 200000)
test.show_best(n = 10)

https://auto.bazos.cz/inzerat/157103354/skoda-octavia-3-20tdi-110kw-dsg-dab-nakup-v-klidu.php
https://auto.bazos.cz/inzerat/157075143/skoda-octavia-3-20tdi-110kw-dsg-plna-zaruka-2-roky-zdarma.php
https://auto.bazos.cz/inzerat/157035970/octavia-scout-4x420tdi-110kwr2018dsgledacc123tiskm.php
https://auto.bazos.cz/inzerat/157035183/octavia-rs-20tsi-230psm2018facecantoncolumbuslane.php
https://auto.bazos.cz/inzerat/156861400/skoda-octavia-3-20tdi-110kw-plna-zaruka-2-roky-zdarma.php
https://auto.bazos.cz/inzerat/156839898/skoda-octavia-3-20tdi-110kw-dsg-plna-zaruka-2-roky-zdarma.php
https://auto.bazos.cz/inzerat/156800462/skoda-octavia-3-16tdi-85kw-plna-zaruka-2-roky-zdarma.php
https://auto.bazos.cz/inzerat/156776866/skoda-octavia-3-fc-14tsi-cng-81kw-dsg-style-koupcr107tkm.php
https://auto.bazos.cz/inzerat/156583245/skoda-octavia-3-16tdi-85kw-dsg-plna-zaruka-2-roky-zdarma.php
https://auto.bazos.cz/inzerat/157066972/skoda-octavia-rs-dsg-fullled-acc-columbus-vyhrsedacky.php
https://auto.bazos

https://auto.bazos.cz/inzerat/155354147/skoda-octavia-3-fc-20tdi-110kw-style-koupcr1majitel2018.php
https://auto.bazos.cz/inzerat/156024216/skoda-octavia-3-rs-liftback.php
https://auto.bazos.cz/inzerat/156879818/multifunkcni-volant-5e0419091ah-skoda-octavia-3-rv-2013.php
https://auto.bazos.cz/inzerat/156879203/ridici-jednotka-abs-5q0907379r-agregat-5q0614517q.php
https://auto.bazos.cz/inzerat/156879000/skoda-octavia-3-greenline-rozvody.php
https://auto.bazos.cz/inzerat/155859328/skoda-octavia-3-rs-20tsi-challenge-combi-manual-facelift.php
https://auto.bazos.cz/inzerat/156867606/skoda-octavia-iii-naraznik.php
https://auto.bazos.cz/inzerat/156853685/prevodovka-6q-dsg-hut-20tsi-147kw-axx-vw-golf-5-gti-152tis.php
https://auto.bazos.cz/inzerat/156752913/skoda-octavia-3-facelift-16tdi-clever-led-navi-alu.php
https://auto.bazos.cz/inzerat/156551382/skoda-octavia-3-combi-16-tdi-81kw-style-vybava.php
https://auto.bazos.cz/inzerat/157127290/skoda-octavia-3-16tdi-182-tkm-servis.php
https://auto.b

https://auto.bazos.cz/inzerat/157063127/palivovy-filtr-octavia-3-5q0127400j.php
https://auto.bazos.cz/inzerat/157063095/skoda-octavia-3fl-20-tsi-140kw.php
https://auto.bazos.cz/inzerat/157062932/palivovy-filtr-5q0127400gj-octavia-3-16-a-20-tdi.php
https://auto.bazos.cz/inzerat/156754521/skoda-octavia-iii-sportline-4x4.php
https://auto.bazos.cz/inzerat/157061689/svetla-na-vozy-skoda-a-vw.php
https://auto.bazos.cz/inzerat/157059985/skoda-octavia-3-bocnice.php
https://auto.bazos.cz/inzerat/157058829/skoda-octavia-3-combi-20-tdi-110-kw.php
https://auto.bazos.cz/inzerat/157058697/renovovany-volant-skoda-2013-2018-octavia-3.php
https://auto.bazos.cz/inzerat/156714436/pravy-blatnik-bila-barva-lf9f-j3j3-skoda-octavia-3-kombi-16.php
https://auto.bazos.cz/inzerat/156718633/tahlo-moturek-steracu-5e1955023a-skoda-octavia-3-2016.php
https://auto.bazos.cz/inzerat/156719202/plato-viko-kufru-5e9867871b-skoda-octavia-3-kombi-2016.php
https://auto.bazos.cz/inzerat/156720264/chromove-hagusy-skoda-octavia

https://auto.bazos.cz/inzerat/157019875/leve-zrcatko-octavia-3-11pin.php
https://auto.bazos.cz/inzerat/157019116/skoda-octavia-3-zadni-naraznik.php
https://auto.bazos.cz/inzerat/157018877/airbag-octavia-3-scala-kodiaq.php
https://auto.bazos.cz/inzerat/157018772/volant-octavia-3-scala-kodiaq-kozeny-volant.php
https://auto.bazos.cz/inzerat/156706392/golf-octavia-rs-superb-zanovni-zimni-alu-kola-17.php
https://auto.bazos.cz/inzerat/156653045/skoda-octavia-3-rs-superb-2-zanovni-zimni-alu-kola-17.php
https://auto.bazos.cz/inzerat/156641348/golf-octavia-superb-passat-zanovni-zimni-alu-kola-17.php
https://auto.bazos.cz/inzerat/157015172/octavia-3-rs-havarovana-pp-castveskere-dilydoklady.php
https://auto.bazos.cz/inzerat/157013813/skoda-octavia-iii-facelift-combi-scout-2018-nahradni-dily.php
https://auto.bazos.cz/inzerat/156702950/predni-naraznik-skoda-octavia-rs-5e0807221aa.php
https://auto.bazos.cz/inzerat/156702447/naraznik-skoda-octavia-3-iii-rs-5e0807221f.php
https://auto.bazos.cz/inzerat

https://auto.bazos.cz/inzerat/156951426/prodam-viko-kufru-na-octavia-3.php
https://auto.bazos.cz/inzerat/155906812/octavia-3-14-tsi-81kw-cng-style-2017-cr-fulllednavi-dph.php
https://auto.bazos.cz/inzerat/156950003/Skoda-Octavia-3-predni-naraznik.php
https://auto.bazos.cz/inzerat/156949506/pouzite-kola-1519565-r165-91h-barum-skoda-oct2.php
https://auto.bazos.cz/inzerat/156949043/skoda-octavia-3-18-tsi-132-kw-rv-2016.php
https://auto.bazos.cz/inzerat/156947964/plechove-disky-skoda-octavia-iii.php
https://auto.bazos.cz/inzerat/156560346/skoda-octavia-iii-style-cr-servis-2017.php
https://auto.bazos.cz/inzerat/156566619/skoda-octavia-iii-facelift-dsg-14tsi-cng-81000km-cr-servis.php
https://auto.bazos.cz/inzerat/156946261/skoda-octavia-3-combi-20-tdi-110kw-dsg-luxury.php
https://auto.bazos.cz/inzerat/156943818/octavia-3-16-tdi.php
https://auto.bazos.cz/inzerat/156942098/predni-kapota-skoda-octavia-3.php
https://auto.bazos.cz/inzerat/156942087/predni-naraznik-skoda-octavia-3-facelift.php
htt

https://auto.bazos.cz/inzerat/156878580/xenon-octavia-3.php
https://auto.bazos.cz/inzerat/156878434/dily-skoda-octavia-3.php
https://auto.bazos.cz/inzerat/156876923/jednotka-radia-octavia-3.php
https://auto.bazos.cz/inzerat/156876747/skoda-octavia-3-dvere-zamek-dveri-sklo-lista-stahovacka.php
https://auto.bazos.cz/inzerat/156875415/SKODA-OCTAVIA-III-16TDI-81KW-odpDPH-2016.php
https://auto.bazos.cz/inzerat/156875352/hybridni-disky-5x112-6jx16-s-octavia-3-vw-seat.php
https://auto.bazos.cz/inzerat/156556085/vstrikovaci-trysky-04l130277aj-skoda-octavia-iii-16tdi-77kw.php
https://auto.bazos.cz/inzerat/156555784/kompletni-led-svetlomet-5e1941015e-skoda-octavia-3-facelift.php
https://auto.bazos.cz/inzerat/156874387/skoda-fabia-octavia-rapid-superb-karoq-kodiaq-narazniky.php
https://auto.bazos.cz/inzerat/156873689/ofuky-oken-skoda-octavia-3-predni-levy-a-pravy.php
https://auto.bazos.cz/inzerat/156872631/skoda-octavia-iii-predni-naraznik-2013-2018.php
https://auto.bazos.cz/inzerat/156872483/sko